# Analyze MLB Matchups

## Module Imports

In [34]:
import pandas as pd
from pandas import DataFrame
import sqlite3
from os import path
from datetime import date
from IPython.core.interactiveshell import InteractiveShell

## Global Variables

In [35]:
# Display all columns of DataFrames
pd.options.display.max_columns = None

# Print all output in a cell not just the last piece of output
InteractiveShell.ast_node_interactivity = "all"

# Location of sports data
DATA_DIR = 'C:\\Users\\Harry\\Documents\\LTCWFF\\ltcwff_files\\data'

# Open database connection
conn = sqlite3.connect(path.join(DATA_DIR, 'mlb.sqlite'))

# Number of games considered "recent"
recent = 10

## Helper Functions

In [36]:
def old_uncompress_table(table):
    rows = []

    for i in range(len(table.index)):
        game = table.iloc[i, :].to_dict()
        row_1 = { (key.split('Home ')[1] if 'Home' in key else key):(val) for (key, val) in game.items() if 'Away' not in key }
        row_2 = { (key.split('Away ')[1] if 'Away' in key else key):(val) for (key, val) in game.items() if 'Home' not in key }
        rows = rows + [row_1, row_2]

    df = pd.DataFrame(rows)
    df['Date'] = df['Date'].astype(str)
    df['Index'] = df[['Date', 'Team']].agg('_'.join, axis = 1)
    df = df.drop(['Date', 'Team'], axis = 1)
    df = df.set_index('Index')
    return df

In [37]:
def uncompress_table(table, team):
    for ind in table.index:
        game = table.loc[ind].to_dict()
        if table.loc[ind, 'Home Team'] == team:
            new_game = {}
            for i in range(len(table.columns)):
                if 2 <= i <= 38:
                    new_game[table.columns[i]] = game[table.columns[i + 1]] if i % 2 == 0 else game[table.columns[i - 1]]
                else:
                    new_game[table.columns[i]] = game[table.columns[i]]
            table.loc[ind] = list(new_game.values())
 
    table.columns = list(table.columns[:2]) + [ ('' if i % 2 == 0 else 'Opponent ') + table.columns[i].split(' ')[-1] for i in range(2, 39) ] + list(table.columns[39:])
    return table


In [38]:
def get_team_table(team):
    table = pd.read_sql(f'''SELECT * FROM new_games WHERE "Away Team" = "{team}" OR "Home Team" = "{team}"''', conn)
    
    for ind in table.index:
        game = table.loc[ind].to_dict()
        if table.loc[ind, 'Home Team'] == team:
            new_game = {}
            for i in range(len(table.columns)):
                if 2 <= i <= 38:
                    new_game[table.columns[i]] = game[table.columns[i + 1]] if i % 2 == 0 else game[table.columns[i - 1]]
                else:
                    new_game[table.columns[i]] = game[table.columns[i]]
            table.loc[ind] = list(new_game.values())
 
    table.columns = list(table.columns[:2]) + [ ('' if i % 2 == 0 else 'Opponent ') + table.columns[i].split(' ')[-1] for i in range(2, 39) ] + list(table.columns[39:])
    return table

## Odds Analysis

In [39]:
# Get today's date in our format
today = date.today()
datef = today.strftime("%Y%m%d")

# Get today's odds as a DataFrame from the database
odds = pd.read_sql(f'''SELECT * FROM odds WHERE date = {datef}''', conn)
odds = odds.set_index('game_id')
odds = odds.drop('Date', axis = 1)
odds

,Away Team,Home Team,Away Spread,Away Spread Odds,Home Spread,Home Spread Odds,Away ML,Home ML,Over,Over Odds,Under,Under Odds,Away 1H ML,Home 1H ML,Away 1H Spread,Away 1H Spread Odds,Home 1H Spread,Home 1H Spread Odds,1H Over,1H Over Odds,1H Under,1H Under Odds
game_id,,,,,,,,,,,,,,,,,,,,,,
ANA202108101,TOR,LAA,1.5,-144,-1.5,120,144,-172,7.5,-118,7.5,-104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PHI202108100,LAD,PHI,-1.5,102,1.5,-122,-164,138,8.5,-118,8.5,-104,-176.0,142.0,-0.5,-124.0,0.5,102.0,4.5,-102.0,4.5,-118.0
PIT202108100,STL,PIT,-1.5,114,1.5,-137,-142,120,9.0,-114,9.0,-106,-154.0,126.0,-0.5,-114.0,0.5,-106.0,4.5,-144.0,4.5,118.0
BAL202108100,DET,BAL,-1.5,115,1.5,-138,-124,106,10.5,-104,10.5,-118,-142.0,116.0,-0.5,-106.0,0.5,-114.0,5.5,-118.0,5.5,-104.0
NYN202108100,WAS,NYM,1.5,-110,-1.5,-110,194,-235,8.5,-108,8.5,-112,172.0,-215.0,1.5,-140.0,-1.5,114.0,4.5,-108.0,4.5,-112.0
BOS202108100,TBR,BOS,1.5,-170,-1.5,140,106,-124,9.5,-118,9.5,-104,112.0,-138.0,0.5,-122.0,-0.5,100.0,5.5,106.0,5.5,-130.0
CLE202108100,OAK,CLE,-1.5,112,1.5,-134,-146,124,9.5,100,9.5,-122,-160.0,130.0,-0.5,-104.0,0.5,-118.0,4.5,-134.0,4.5,110.0
ATL202108100,CIN,ATL,-1.5,146,1.5,-178,-104,-112,9.5,-110,9.5,-110,-114.0,-106.0,0.5,-150.0,-0.5,122.0,5.5,100.0,5.5,-122.0
CHN202108102,MIL,CHI,-1.5,108,1.5,-130,-154,130,8.0,-112,8.0,-108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# Loop through each row of the odds DataFrame and append analysis for each game
for ind in odds.index:
    away_team = odds.loc[ind, 'Away Team']
    home_team = odds.loc[ind, 'Home Team']
    
    # Get the player's table for that prop
    #away = uncompress_table(pd.read_sql(f'''SELECT * FROM new_games WHERE "Away Team" = "{away_team}" OR "Home Team" = "{away_team}"''', conn), home_team)
    #home = uncompress_table(pd.read_sql(f'''SELECT * FROM new_games WHERE "Away Team" = "{home_team}" OR "Home Team" = "{home_team}"''', conn), away_team)
    away = get_team_table(away_team)
    recent_away = away.tail(recent)
    home = get_team_table(home_team)
    recent_home = home.tail(recent)

    odds.loc[ind, 'Away Games'] = len(away)
    odds.loc[ind, 'Away Wins'] = len(away.loc[away['R'] > away['Opponent R']])
    odds.loc[ind, 'Away Losses'] = len(away.loc[away['R'] < away['Opponent R']])
    odds.loc[ind, 'Away Winning Pct'] = odds.loc[ind, 'Away Wins'] / odds.loc[ind, 'Away Games']
    odds.loc[ind, 'Home Games'] = len(home)
    odds.loc[ind, 'Home Wins'] = len(home.loc[home['R'] > home['Opponent R']])
    odds.loc[ind, 'Home Losses'] = len(home.loc[home['R'] < home['Opponent R']])
    odds.loc[ind, 'Home Winning Pct'] = odds.loc[ind, 'Home Wins'] / odds.loc[ind, 'Home Games']
    odds.loc[ind, 'Away Mean Margin'] = (away['R'] - away['Opponent R']).mean()
    odds.loc[ind, 'Away Median Margin'] = (away['R'] - away['Opponent R']).median()
    odds.loc[ind, 'Away Spread "Cover" Pct'] = (away['Opponent R'] - away['R'] < odds.loc[ind, 'Away Spread']).mean()
    odds.loc[ind, 'Home Mean Margin'] = (home['R'] - home['Opponent R']).mean()
    odds.loc[ind, 'Home Median Margin'] = (home['R'] - home['Opponent R']).median()
    odds.loc[ind, 'Home Spread "Cover" Pct'] = (home['Opponent R'] - home['R'] < odds.loc[ind, 'Home Spread']).mean()
    odds.loc[ind, 'Away Mean Total'] = (away['R'] + away['Opponent R']).mean()
    odds.loc[ind, 'Away Median Total'] = (away['R'] + away['Opponent R']).median()
    odds.loc[ind, 'Away Total "Over" Pct'] = (away['Opponent R'] + away['R'] > odds.loc[ind, 'Over']).mean()
    odds.loc[ind, 'Home Mean Total'] = (home['R'] + home['Opponent R']).mean()
    odds.loc[ind, 'Home Median Total'] = (home['R'] + home['Opponent R']).median()
    odds.loc[ind, 'Home Total "Over" Pct'] = (home['Opponent R'] + home['R'] > odds.loc[ind, 'Over']).mean()
    odds.loc[ind, 'Recent Away Wins'] = len(recent_away.loc[recent_away['R'] > recent_away['Opponent R']])
    odds.loc[ind, 'Recent Away Losses'] = len(recent_away.loc[recent_away['R'] < recent_away['Opponent R']])
    odds.loc[ind, 'Recent Away Winning Pct'] = odds.loc[ind, 'Recent Away Wins'] / recent
    odds.loc[ind, 'Recent Home Wins'] = len(recent_home.loc[recent_home['R'] > recent_home['Opponent R']])
    odds.loc[ind, 'Recent Home Losses'] = len(recent_home.loc[recent_home['R'] < recent_home['Opponent R']])
    odds.loc[ind, 'Recent Home Winning Pct'] = odds.loc[ind, 'Recent Home Wins'] / recent
    odds.loc[ind, 'Recent Away Mean Margin'] = (recent_away['R'] - recent_away['Opponent R']).mean()
    odds.loc[ind, 'Recent Away Median Margin'] = (recent_away['R'] - recent_away['Opponent R']).median()
    odds.loc[ind, 'Recent Away Spread "Cover" Pct'] = (recent_away['Opponent R'] - recent_away['R'] < odds.loc[ind, 'Away Spread']).mean()
    odds.loc[ind, 'Recent Home Mean Margin'] = (recent_home['R'] - recent_home['Opponent R']).mean()
    odds.loc[ind, 'Recent Home Median Margin'] = (recent_home['R'] - recent_home['Opponent R']).median()
    odds.loc[ind, 'Recent Home Spread "Cover" Pct'] = (recent_home['Opponent R'] - recent_home['R'] < odds.loc[ind, 'Home Spread']).mean()
    odds.loc[ind, 'Recent Away Mean Total'] = (recent_away['R'] + recent_away['Opponent R']).mean()
    odds.loc[ind, 'Recent Away Median Total'] = (recent_away['R'] + recent_away['Opponent R']).median()
    odds.loc[ind, 'Recent Away Total "Over" Pct'] = (recent_away['Opponent R'] + recent_away['R'] > odds.loc[ind, 'Over']).mean()
    odds.loc[ind, 'Recent Home Mean Total'] = (recent_home['R'] + recent_home['Opponent R']).mean()
    odds.loc[ind, 'Recent Home Median Total'] = (recent_home['R'] + recent_home['Opponent R']).median()
    odds.loc[ind, 'Recent Home Total "Over" Pct'] = (recent_home['Opponent R'] + recent_home['R'] > odds.loc[ind, 'Over']).mean()

    #Wins	Losses	Average Spread	Median Margin	Covers	Not Covers	Pushes	Cover Pct	Average Total	Median Points	Overs	Unders	Total Pushes	Over Pct	1H Average Spread	1H Median Margin	1H Covers	1H Not Covers	1H Pushes	1H Cover Pct	1H Average Total	1H Median Points	1H Overs	1H Unders	1H Total Pushes	1H Over Pct	2H Average Spread	2H Median Margin	2H Covers	2H Not Covers	2H Pushes	2H Cover Pct	2H Average Total	2H Median Points	2H Overs	2H Unders	2H Total Pushes	2H Over Pct	1Q Average Spread	1Q Median Margin	1Q Covers	1Q Not Covers	1Q Pushes	1Q Cover Pct	1Q Average Total	1Q Median Points	1Q Overs	1Q Unders	1Q Total Pushes	1Q Over Pct	2Q Average Spread	2Q Median Margin	2Q Covers	2Q Not Covers	2Q Pushes	2Q Cover Pct	2Q Average Total	2Q Median Points	2Q Overs	2Q Unders	2Q Total Pushes	2Q Over Pct	3Q Average Spread	3Q Median Margin	3Q Covers	3Q Not Covers	3Q Pushes	3Q Cover Pct	3Q Average Total	3Q Median Points	3Q Overs	3Q Unders	3Q Total Pushes	3Q Over Pct	4Q Average Spread	4Q Median Margin	4Q Covers	4Q Not Covers	4Q Pushes	4Q Cover Pct	4Q Average Total	4Q Median Points	4Q Overs	4Q Unders	4Q Total Pushes	4Q Over Pct

odds

,Away Team,Home Team,Away Spread,Away Spread Odds,Home Spread,Home Spread Odds,Away ML,Home ML,Over,Over Odds,Under,Under Odds,Away 1H ML,Home 1H ML,Away 1H Spread,Away 1H Spread Odds,Home 1H Spread,Home 1H Spread Odds,1H Over,1H Over Odds,1H Under,1H Under Odds,Away Games,Away Wins,Away Losses,Away Winning Pct,Home Games,Home Wins,Home Losses,Home Winning Pct,Away Mean Margin,Away Median Margin,"Away Spread ""Cover"" Pct",Home Mean Margin,Home Median Margin,"Home Spread ""Cover"" Pct",Away Mean Total,Away Median Total,"Away Total ""Over"" Pct",Home Mean Total,Home Median Total,"Home Total ""Over"" Pct",Recent Away Wins,Recent Away Losses,Recent Away Winning Pct,Recent Home Wins,Recent Home Losses,Recent Home Winning Pct,Recent Away Mean Margin,Recent Away Median Margin,"Recent Away Spread ""Cover"" Pct",Recent Home Mean Margin,Recent Home Median Margin,"Recent Home Spread ""Cover"" Pct",Recent Away Mean Total,Recent Away Median Total,"Recent Away Total ""Over"" Pct",Recent Home Mean Total,Recent Home Median Total,"Recent Home Total ""Over"" Pct"
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ANA202108101,TOR,LAA,1.5,-144,-1.5,120,144,-172,7.5,-118,7.5,-104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0,54.0,48.0,0.529412,105.0,52.0,53.0,0.495238,1.049020,1.0,0.647059,-0.409524,-1.0,0.323810,9.303922,9.0,0.676471,9.742857,10.0,0.704762,6.0,4.0,0.6,5.0,5.0,0.5,2.4,2.5,0.8,-1.0,0.0,0.2,7.8,7.5,0.5,7.6,8.0,0.6
PHI202108100,LAD,PHI,-1.5,102,1.5,-122,-164,138,8.5,-118,8.5,-104,-176.0,142.0,-0.5,-124.0,0.5,102.0,4.5,-102.0,4.5,-118.0,107.0,64.0,43.0,0.598131,105.0,52.0,53.0,0.495238,1.542056,1.0,0.476636,-0.190476,-1.0,0.638095,8.925234,8.0,0.439252,9.295238,8.0,0.457143,5.0,5.0,0.5,5.0,5.0,0.5,1.6,0.0,0.3,-0.4,0.0,0.6,8.0,8.0,0.4,10.2,8.5,0.5
PIT202108100,STL,PIT,-1.5,114,1.5,-137,-142,120,9.0,-114,9.0,-106,-154.0,126.0,-0.5,-114.0,0.5,-106.0,4.5,-144.0,4.5,118.0,105.0,53.0,52.0,0.504762,105.0,40.0,65.0,0.380952,-0.352381,1.0,0.371429,-1.428571,-2.0,0.476190,8.390476,8.0,0.380952,8.742857,8.0,0.352381,6.0,4.0,0.6,4.0,6.0,0.4,0.1,1.0,0.4,-2.5,-3.0,0.4,8.5,8.5,0.3,10.1,10.0,0.6
BAL202108100,DET,BAL,-1.5,115,1.5,-138,-124,106,10.5,-104,10.5,-118,-142.0,116.0,-0.5,-106.0,0.5,-114.0,5.5,-118.0,5.5,-104.0,108.0,51.0,57.0,0.472222,104.0,37.0,67.0,0.355769,-0.462963,-1.0,0.324074,-1.317308,-1.5,0.500000,9.296296,8.0,0.342593,9.625000,9.0,0.403846,4.0,6.0,0.4,6.0,4.0,0.6,-0.1,-1.0,0.3,0.0,1.0,0.7,11.5,8.0,0.4,8.8,8.0,0.1
NYN202108100,WAS,NYM,1.5,-110,-1.5,-110,194,-235,8.5,-108,8.5,-112,172.0,-215.0,1.5,-140.0,-1.5,114.0,4.5,-108.0,4.5,-112.0,105.0,49.0,56.0,0.466667,104.0,55.0,49.0,0.528846,-0.257143,-1.0,0.580952,-0.067308,1.0,0.307692,9.019048,8.0,0.400000,7.701923,7.0,0.326923,4.0,6.0,0.4,4.0,6.0,0.4,-0.9,-1.0,0.6,-2.5,-2.5,0.0,9.5,9.0,0.6,7.9,8.5,0.5
BOS202108100,TBR,BOS,1.5,-170,-1.5,140,106,-124,9.5,-118,9.5,-104,112.0,-138.0,0.5,-122.0,-0.5,100.0,5.5,106.0,5.5,-130.0,106.0,64.0,42.0,0.603774,107.0,63.0,44.0,0.588785,1.056604,1.0,0.754717,0.448598,1.0,0.401869,8.962264,9.0,0.415094,9.457944,9.0,0.439252,7.0,3.0,0.7,4.0,6.0,0.4,3.1,2.5,0.9,-1.6,-1.0,0.2,9.3,9.5,0.5,8.6,8.5,0.3
CLE202108100,OAK,CLE,-1.5,112,1.5,-134,-146,124,9.5,100,9.5,-122,-160.0,130.0,-0.5,-104.0,0.5,-118.0,4.5,-134.0,4.5,110.0,107.0,60.0,47.0,0.560748,102.0,51.0,51.0,0.500000,0.485981,1.0,0.420561,-0.294118,0.0,0.647059,8.280374,8.0,0.336449,8.941176,9.0,0.401961,5.0,5.0,0.5,4.0,6.0,0.4,1.3,0.5,0.5,-0.9,-1.0,0.6,7.1,7.0,0.3,9.9,9.0,0.4
ATL202108100,CIN,ATL,-1.5,146,1.5,-178,-104,-112,9.5,-110,9.5,-110,-114.0,-106.0,0.5,-150.0,-0.5,122.0,5.5,100.0,5.5,-122.0,106.0,56.0,50.0,0.528302,106.0,52.0,54.0,0.490566,0.009434,1.0,0.358491,0.462264,-1.0,0.679245,9.820755,9.0,0.462264,9.028302,8.0,0.386792,7.0,3.0,0.7,5.0,5.0,0.5,1.9,2.5,0.6,2.3,0.5,0.9,10.3,10.5,0.6,7.9,6.0,0.3
CHN202108102,MIL,CHI,-1.5,108,1.5,-130,-154,130,8.0,-112,8.0,-108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,63.0,43.0,0.594340,107.0,51.0,56.0,0.476636,0.830189,1.0,0.443396,-0.2

In [41]:
odds.columns

Index(['Away Team', 'Home Team', 'Away Spread', 'Away Spread Odds',
       'Home Spread', 'Home Spread Odds', 'Away ML', 'Home ML', 'Over',
       'Over Odds', 'Under', 'Under Odds', 'Away 1H ML', 'Home 1H ML',
       'Away 1H Spread', 'Away 1H Spread Odds', 'Home 1H Spread',
       'Home 1H Spread Odds', '1H Over', '1H Over Odds', '1H Under',
       '1H Under Odds', 'Away Games', 'Away Wins', 'Away Losses',
       'Away Winning Pct', 'Home Games', 'Home Wins', 'Home Losses',
       'Home Winning Pct', 'Away Mean Margin', 'Away Median Margin',
       'Away Spread "Cover" Pct', 'Home Mean Margin', 'Home Median Margin',
       'Home Spread "Cover" Pct', 'Away Mean Total', 'Away Median Total',
       'Away Total "Over" Pct', 'Home Mean Total', 'Home Median Total',
       'Home Total "Over" Pct', 'Recent Away Wins', 'Recent Away Losses',
       'Recent Away Winning Pct', 'Recent Home Wins', 'Recent Home Losses',
       'Recent Home Winning Pct', 'Recent Away Mean Margin',
       'Recent Aw

In [51]:
columns = ['Team', 'Spread', 'Spread Odds', 'Mean Margin', 'Median Margin', 'Spread "Cover" Pct', 'ML', 'Winning Pct', 'Over', 'Over Odds', 'Under', 'Under Odds', 'Mean Total', 'Median Total', 'Total "Over" Pct']
shared_columns = ['Over', 'Over Odds', 'Under', 'Under Odds']

# Loop through each row of the odds DataFrame and append analysis for each game
for ind in odds.index:
    home_row = { col:(odds.loc[ind, col] if col in shared_columns else odds.loc[ind, f'Home {col}']) for col in columns }
    away_row = { col:(odds.loc[ind, col] if col in shared_columns else odds.loc[ind, f'Away {col}']) for col in columns }
    df = DataFrame([home_row, away_row]).set_index('Team')
    df

,Spread,Spread Odds,Mean Margin,Median Margin,"Spread ""Cover"" Pct",ML,Winning Pct,Over,Over Odds,Under,Under Odds,Mean Total,Median Total,"Total ""Over"" Pct"
Team,,,,,,,,,,,,,,
LAA,-1.5,120,-0.409524,-1.0,0.323810,-172,0.495238,7.5,-118,7.5,-104,9.742857,10.0,0.704762
TOR,1.5,-144,1.049020,1.0,0.647059,144,0.529412,7.5,-118,7.5,-104,9.303922,9.0,0.676471


,Spread,Spread Odds,Mean Margin,Median Margin,"Spread ""Cover"" Pct",ML,Winning Pct,Over,Over Odds,Under,Under Odds,Mean Total,Median Total,"Total ""Over"" Pct"
Team,,,,,,,,,,,,,,
PHI,1.5,-122,-0.190476,-1.0,0.638095,138,0.495238,8.5,-118,8.5,-104,9.295238,8.0,0.457143
LAD,-1.5,102,1.542056,1.0,0.476636,-164,0.598131,8.5,-118,8.5,-104,8.925234,8.0,0.439252


,Spread,Spread Odds,Mean Margin,Median Margin,"Spread ""Cover"" Pct",ML,Winning Pct,Over,Over Odds,Under,Under Odds,Mean Total,Median Total,"Total ""Over"" Pct"
Team,,,,,,,,,,,,,,
PIT,1.5,-137,-1.428571,-2.0,0.476190,120,0.380952,9.0,-114,9.0,-106,8.742857,8.0,0.352381
STL,-1.5,114,-0.352381,1.0,0.371429,-142,0.504762,9.0,-114,9.0,-106,8.390476,8.0,0.380952


,Spread,Spread Odds,Mean Margin,Median Margin,"Spread ""Cover"" Pct",ML,Winning Pct,Over,Over Odds,Under,Under Odds,Mean Total,Median Total,"Total ""Over"" Pct"
Team,,,,,,,,,,,,,,
BAL,1.5,-138,-1.317308,-1.5,0.500000,106,0.355769,10.5,-104,10.5,-118,9.625000,9.0,0.403846
DET,-1.5,115,-0.462963,-1.0,0.324074,-124,0.472222,10.5,-104,10.5,-118,9.296296,8.0,0.342593


,Spread,Spread Odds,Mean Margin,Median Margin,"Spread ""Cover"" Pct",ML,Winning Pct,Over,Over Odds,Under,Under Odds,Mean Total,Median Total,"Total ""Over"" Pct"
Team,,,,,,,,,,,,,,
NYM,-1.5,-110,-0.067308,1.0,0.307692,-235,0.528846,8.5,-108,8.5,-112,7.701923,7.0,0.326923
WAS,1.5,-110,-0.257143,-1.0,0.580952,194,0.466667,8.5,-108,8.5,-112,9.019048,8.0,0.400000


...

,Spread,Spread Odds,Mean Margin,Median Margin,"Spread ""Cover"" Pct",ML,Winning Pct,Over,Over Odds,Under,Under Odds,Mean Total,Median Total,"Total ""Over"" Pct"
Team,,,,,,,,,,,,,,
HOU,-1.5,105,1.405660,1.0,0.490566,-184,0.603774,8.5,-120,8.5,-102,9.537736,9.5,0.622642
COL,1.5,-126,-0.537736,-1.0,0.594340,154,0.433962,8.5,-120,8.5,-102,9.198113,9.0,0.518868


,Spread,Spread Odds,Mean Margin,Median Margin,"Spread ""Cover"" Pct",ML,Winning Pct,Over,Over Odds,Under,Under Odds,Mean Total,Median Total,"Total ""Over"" Pct"
Team,,,,,,,,,,,,,,
SFG,-1.5,100,1.133333,1.0,0.476190,-210,0.628571,8.0,-106,8.0,-114,8.580952,8.0,0.428571
ARI,1.5,-120,-1.575472,-1.0,0.509434,176,0.311321,8.0,-106,8.0,-114,9.839623,9.0,0.556604


,Spread,Spread Odds,Mean Margin,Median Margin,"Spread ""Cover"" Pct",ML,Winning Pct,Over,Over Odds,Under,Under Odds,Mean Total,Median Total,"Total ""Over"" Pct"
Team,,,,,,,,,,,,,,
LAA,1.5,-122,-0.409524,-1.0,0.580952,132,0.495238,7.5,-118,7.5,-104,9.742857,10.0,0.704762
TOR,-1.5,102,1.049020,1.0,0.470588,-156,0.529412,7.5,-118,7.5,-104,9.303922,9.0,0.676471


,Spread,Spread Odds,Mean Margin,Median Margin,"Spread ""Cover"" Pct",ML,Winning Pct,Over,Over Odds,Under,Under Odds,Mean Total,Median Total,"Total ""Over"" Pct"
Team,,,,,,,,,,,,,,
SEA,-1.5,-102,-0.528302,1.0,0.311321,-225,0.528302,8.0,-104,8.0,-118,8.981132,9.0,0.537736
TEX,1.5,-118,-1.085714,-2.0,0.495238,188,0.361905,8.0,-104,8.0,-118,8.800000,8.0,0.495238


,Spread,Spread Odds,Mean Margin,Median Margin,"Spread ""Cover"" Pct",ML,Winning Pct,Over,Over Odds,Under,Under Odds,Mean Total,Median Total,"Total ""Over"" Pct"
Team,,,,,,,,,,,,,,
SDP,-1.5,-118,0.861111,1.0,0.425926,-230,0.564815,8.0,-106,8.0,-114,8.675926,8.0,0.435185
MIA,1.5,-102,-0.019048,-1.0,0.628571,190,0.419048,8.0,-106,8.0,-114,7.809524,7.0,0.342857
